---
title: Spatial Interaction with Spatial Dependence
jupyter: conda-env-nersa25-py
---

In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
import scipy
from formulaic import Formula
from geosnap import DataStore
from geosnap import io as gio
from libpysal.graph import Graph
from shapely import LineString
from spreg import GMM_Error, GM_Lag
%load_ext watermark
%watermark -a 'eli knaap'

What if our flows are not independent?

Like any regression, a critical assumption in spatial interaction models
is that observations are *independent* from one another. And like any model
using spatial data, the model is misspecified if residuals are spatially
autocorrelated (indicating the input data fail the independence criterion). We
can use spatial econometric approaches to handle this situation, albeit with
some minor modifications because

- autocorrelation may come from origins, destinations, or both
- we need to approximate the data using a log-linear model instead of proper Poisson

Approaches for estimating spatial lag models are described in
@lesage2007KnowledgeSpillovers and @lesage2008SpatialEconometric while error
models are described by @fischer2008ModelingSpatial, the latter two of which use
conventional estimation techniques with specialized $W$ matrices based on the
notion of neighboring origins or neighboring destinations. We explore how to
conduct these analyses below. For further background, consult
@fischer2008ModelingSpatial, @lesage2007KnowledgeSpillovers,
@lesage2008SpatialEconometric, @lesage2010SpatialEconometric,
@lesage2013SpatialInteraction, @lesage2014WhatRegional,
@thomas-agnan2014SpatialEconometric, @griffith2017SpatialAutocorrelation and
@ord1975EstimationMethods.

## Spatial Econometric Models

In the following example we will focus on the spatial interaction specification
of two workhorse models in spatial econometrics: the "spatial lag" and "spatial
error" models. Following the log-linear specification from the prior section,
these are given by

### Spatial Lag

$$ \log(F_{ij} + 
\delta) =  \log(\kappa) + \rho W\log(F_{ij}+\delta) + \alpha \log(O_i)  + \beta \log(D_j) + \gamma d_{ij} + \epsilon_{ij} $$ {#eq-interaction-spatlag}

### Spatial Error

$$
\begin{gathered}
\log(F_{ij} + 
\delta) = \log(\kappa) +  \alpha \log(O_i)  + \beta \log(D_j) + \gamma d_{ij} + u \\
u = \lambda Wu +\epsilon_{ij}
\end{gathered}
$$  {#eq-interaction-spaterr}


Note there's a sizeable and growing literature focused on the appropriateness of
different estimation techniques for count-based models, particularly in the
gravity model context
[@manning2001EstimatingLog; @santossilva2010ExistenceMaximum; @santossilva2011FurtherSimulation; @silva2006LogGravity].
For the purpose of this workshop it's sufficient to say that log-linear models
induce a certain level of bias--and it's important to be aware. However flow
models also display empirical residual autocorrelation, so nonlinear but
nonspatial models *also* induce bias (and it's hard to estimate nonlinear
spatial models). So here we accept one bias in favor of the other for the sake
of demonstration and concern for spatial effects. Consult the literature for a
deeper dive.

## Data Preparation

We will follow the same data processing steps as in the previous sections,
collecting data for Washington D.C. and converting it into a Graph of flows,
then merging with additional data from the Census.

In [ ]:
datasets = DataStore()

dc = gio.get_acs(datasets, state_fips="11", years=2021, level="tract")

dc_flows = pd.read_csv(
    "https://lehd.ces.census.gov/data/lodes/LODES8/dc/od/dc_od_main_JT00_2022.csv.gz",
    converters={"w_geocode": str, "h_geocode": str},
    low_memory=False,
    encoding="latin1",
)
dc_flows["w_tr_geocode"] = dc_flows["w_geocode"].str[:11]
dc_flows["h_tr_geocode"] = dc_flows["h_geocode"].str[:11]
dc_flows = dc_flows[["w_geocode", "h_geocode", "w_tr_geocode", "h_tr_geocode", "S000"]]
dc_flows = (
    dc_flows.groupby(["w_tr_geocode", "h_tr_geocode"])["S000"].sum().reset_index()
)

dc_flow_graph = Graph.from_adjacency(
    adjacency=dc_flows,
    focal_col="h_tr_geocode",
    neighbor_col="w_tr_geocode",
    weight_col="S000",
)

dc = dc.set_index("geoid")

# for our dataset we want the full dense matrix
dc_interaction = pd.Series(
    dc_flow_graph.sparse.toarray().reshape(-1),
    index=pd.MultiIndex.from_product(
        [dc_flow_graph.unique_ids, dc_flow_graph.unique_ids.rename("neighbor")]
    ),
).rename("weight")

dc_interaction = dc_interaction.reset_index()

# first merge origin attributes
dc_interaction = dc_interaction.merge(
    dc.drop(columns=["geometry"]), left_on="focal", right_index=True, how="left"
)

# now merge destination attributes
dc_interaction = dc_interaction.merge(
    dc.drop(columns=["geometry"]),
    left_on="neighbor",
    right_index=True,
    how="left",
    suffixes=["_origin", "_destination"],
)

## A Confluence of `Graphs`

What's "near" to a *flow*?

### Distance Graph

As with the conventional models in the previous section, we need the distance between each OD pair as a variable for our model. Again we keep only tracts in the dataframe in our flow graph (origins), then get distance between observations with no decay using a `Graph`

In [ ]:
dc = dc.to_crs(dc.estimate_utm_crs())

dc = dc[dc.index.isin(dc_flow_graph.unique_ids)]

dc_dist = Graph.build_distance_band(
    dc.set_geometry(dc.centroid), threshold=1e20, binary=False, alpha=1
)

In [ ]:
dc_dist.summary()

In [ ]:
dc_dist.adjacency

In [ ]:
# subset the distance graph by the travel graph (remove destinations we dont need)
# but this resets weights to 1
dc_dist_adj = dc_dist.intersection(dc_flow_graph).adjacency

# update with the old values
dc_dist_adj.update(dc_dist.adjacency)

now create our dataset using the dense matrix 'melted' down into a vector

In [ ]:
dc_interaction["distance"] = dc_dist.sparse.toarray().reshape(-1)

dc_interaction['weight'] = dc_interaction['weight'].astype(int)

### Contiguity Graph

Now we need to relate the origin and destination observations together. To keep
things simple, we consider the standard contiguity graph.

In [ ]:
contg = Graph.build_contiguity(dc)

In [ ]:
contg

Imagine you had a flow that moved from north to south like the map below. The
'neighborhood' of this flow might be the tracts surrounding the origin in the
north, those surrounding the destination in the south, or a combination thereof.

In [ ]:
focus_tracts = ["11001007703", "11001001804"]

contg.explore(dc.centroid)

m = dc.explore(tiles="CartoDB Positron", tooltip=["geoid"])
contg.explore(
    dc,
    m=m,
    focal=focus_tracts,
    edge_kws=dict(color="red"),
    node_kws=dict(style_kwds=dict(radius=4, color="yellow")),
)
l = gpd.GeoDataFrame(geometry=[LineString(dc.loc[focus_tracts].geometry.centroid.get_coordinates()[['x','y']].values)], crs=dc.crs)
l.explore(m=m, color='red', style_kwds={'weight':6})

### Spatial Graphs for Origin-Destination Flows

the `contg` Graph encodes flows as neighbors if the origin tracts share a
border. But we need to multiply that graph to get it into the correct dimensions
to match our flow data. Following @lesage2008SpatialEconometric and
@fischer2008ModelingSpatial we do this via a
[*Kronecker* product](https://en.wikipedia.org/wiki/Kronecker_product) between
our flow and contiguity graphs to create the graph ($W$) used in the model.

![Kronecker Product](../img/kron.png)

$G_{flow} \otimes G_{cont}$, where $\otimes$ is the Kronecker product of the
flow graph and contiguity graphs that defines connectivity between origin and
destination observations.

In this case observations are neighbors if:

- there is a flow between o and d
- if o_i shares a border with o_j
- three distinct possibilities depending on how the flow graph is ordered
  - origin-centric weights
  - destination-centric weights
  - OD-centric weights (union or sum of oW and dW)

To do this in code, we use `scipy` to take the Kronecker product of the two
`Graph`s, then re-instantiate a new one.

In [ ]:
kg = Graph.from_sparse(scipy.sparse.kron(dc_flow_graph.transform("b").sparse, contg.sparse))

our new graph now as the same length as our observation vector

In [ ]:
kg.n

In [ ]:
dc_interaction.shape[0]

In [ ]:
kg.pct_nonzero

In [ ]:
contg.pct_nonzero

In [ ]:
dc_flow_graph.pct_nonzero

since our original `Graph` has the origin as its focal observation, this is an origin-centric ODW ($^oW$), so to get the destination-centric weights
($^dW$) you'd do the transpose of the OD matrix (flow graph) first
[@lesage2008SpatialEconometric].

In [ ]:
kgd = Graph.from_sparse(scipy.sparse.kron(dc_flow_graph.transform("b").sparse.transpose(), contg.sparse))

In [ ]:
kgd.pct_nonzero

row-standardize both origin and destination versions

In [ ]:
kg = kg.transform('r')

In [ ]:
kgd = kgd.transform('r')

`spreg` will only treat the Graph as a matrix, so the ordering of the sparse
representation is all that matters, not the indices/labels; i.e. the Graph has
the correct shape and order even though the indices of the Graph are different
than those of the observations

## Main Model Specification

> "Note in some cases yij = 0, indicating the absence of flows from i to j. This
> leads to the so-called zero problem, since the logarithm then is undefined.
> There are several pragmatic solutions to this problem, with adding a small
> constant to the zero elements of [yij ] being widely used. Here we added
> 0.08." [@fischer2008ModelingSpatial]

that gives our $y$ variable a distribution like this

In [ ]:
dc_interaction.weight.replace(0,0.08).apply(np.log).hist()

Another common transformation is to add 1 to every observation and take the log
of *that*, i.e. take $log(x+1)$ (this is setting $\delta=1$ in
@eq-interaction-log-linear).

In [ ]:
dc_interaction.weight.apply(np.log1p).hist()

we specify a log-linear model using `formulaic` to generate our $y$ and $X$ matrices, then pass these to different kinds of spatial econometric models

In [ ]:
form = "np.log1p(weight) ~ 1+ np.log1p(n_total_pop_origin) + np.log1p(median_household_income_origin) + np.log1p(p_nonhisp_black_persons_origin) + np.log1p(n_total_pop_destination) + np.log1p(median_household_income_destination) + np.log1p(p_nonhisp_black_persons_destination) + np.log1p(distance)"

In [ ]:
f = Formula(form)

# mean-impute missing values for convenience
y, x = f.get_model_matrix(
    dc_interaction.fillna(dc_interaction.mean(numeric_only=True))
)

In [ ]:
y

## Spatial Lag

### Origin centric weights

In [ ]:
flow_lag = GM_Lag(y=y, x=x, w=kg, robust='white')

In [ ]:
#| column: page
print(flow_lag.summary)

In [ ]:
flow_lag.output

In [ ]:
pd.Series(flow_lag.u.flatten()).hist()

### Destination-centric weights


In [ ]:
dest_flow_lag = GM_Lag(y=y, x=x,  w=kgd)

In [ ]:
#| column: page
print(dest_flow_lag.summary)

In [ ]:
dest_flow_lag.output

### OD-centric weights

one "OD-graph" could be the union of the two; a flow is 'neighbors' with another flow if it is contiguous with *either* origin or destination points

In [ ]:
kg_od = kg.transform('b').union(kgd.transform('b'))

In [ ]:
kg_od = kg_od.transform('r')

In [ ]:
kg_od.pct_nonzero

In [ ]:
od_flow_lag = GM_Lag(y=y, x=x, w=kg_od)

In [ ]:
#| column: page

print(od_flow_lag.summary)

In [ ]:
od_flow_lag.output

Instead, we could take the sum of the two graphs, in which case you are
neighbors when contiguous with *either* origin or destination points (same
cardinalities as above), but the strength of the weight is 2x if you neighbor
*both* origin and destination.

In [ ]:
kg_od = Graph.from_sparse(kg.transform('b').sparse + kgd.transform('b').sparse)
kg_od = kg_od.transform('r')

In [ ]:
#| column: page

print(od_flow_lag.summary)

In [ ]:
od_flow_lag.output

## Spatial Error

the error models take a really long time to estimate

### Origin-Centric

In [ ]:
flow_error_origin = GMM_Error(y=y, x=x, w=kg)

In [ ]:
#| column: page

print(flow_error_origin.summary)

In [ ]:
flow_error_origin.output

### Destination-centric

In [ ]:
flow_error_dest = GMM_Error(y=y, x=x, w=kgd)

In [ ]:
#| column: page

print(flow_error_dest.summary)

In [ ]:
flow_error_dest.output

### OD-Centric

In [ ]:
flow_error_od = GMM_Error(y=y, x=x, w=kg_od)

In [ ]:
#| column: page

print(flow_error_od.summary)

In [ ]:
flow_error_od.output

::: callout-note :::

As an alternative to the spatial econometric specifications illustrated above,
@liao2025DataDrivenApproach recently described a different model that
incorporates the "intervening opportunities" and "competing destinations"
frameworks discussed in the spatial interaction literature by incorporating two
additional terms $A_i$ and $A_j$ which represent accessibility measures at the
origin and destination locations, respectively. In spatial econometric parlance,
this approach is equivalent to a spatial lag of X (SLX) model with terms that
include both origin-centric and destination-centric lagged X variables
[@lesage2016SpatialRegressionBased].

:::


## References
